# Monte Carlo Methods

### Part 0: Explore BlackjackEnv

Use the code cell below to create an instance of the [Blackjack](https://github.com/openai/gym/blob/master/gym/envs/toy_text/blackjack.py) environment.

In [ ]:
import gym
env = gym.make('Blackjack-v0')

Each state is a 3-tuple of:
- the player's current sum $\in \{0, 1, \ldots, 31\}$,
- the dealer's face up card $\in \{1, \ldots, 10\}$, and
- whether or not the player has a usable ace (`no` $=0$, `yes` $=1$).

The agent has two potential actions:

```
    STICK = 0
    HIT = 1
```
Verify this by running the code cell below.

In [ ]:
print(env.observation_space)
print(env.action_space)

Execute the code cell below to play Blackjack with a random policy.  

(_The code currently plays Blackjack three times - feel free to change this number, or to run the cell multiple times.  The cell is designed for you to get some experience with the output that is returned as the agent interacts with the environment._)

In [ ]:
for i_episode in range(3):
    state = env.reset()
    while True:
        print(state)
        action = env.action_space.sample()
        state, reward, done, info = env.step(action)
        if done:
            print(state)
            print('End game! Reward: ', reward)
            print('You won :)\n') if reward > 0 else print('You lost :(\n')
            break

### Part 1: MC Prediction: State Values

In this section, you will write your own implementation of MC prediction (for estimating the state-value function).

We will begin by investigating a policy where the player always sticks if the sum of her cards exceeds 18.  The function `generate_episode_from_limit` samples an episode using this policy. 

The function accepts as **input**:
- `bj_env`: This is an instance of OpenAI Gym's Blackjack environment.

It returns as **output**:
- `episode`: This is a list of (state, action, reward) tuples (of tuples) and corresponds to $(S_0, A_0, R_1, \ldots, S_{T-1}, A_{T-1}, R_{T})$, where $T$ is the final time step.  In particular, `episode[i]` returns $(S_i, A_i, R_{i+1})$, and `episode[i][0]`, `episode[i][1]`, and `episode[i][2]` return $S_i$, $A_i$, and $R_{i+1}$, respectively.

In [ ]:
def generate_episode_from_limit(bj_env):
    episode = []
    state = bj_env.reset()
    while True:
        action = 0 if state[0] > 18 else 1
        next_state, reward, done, info = bj_env.step(action)
        episode.append((state, action, reward))
        state = next_state
        if done:
            break
    return episode

Execute the code cell below to play Blackjack with the policy. 

(*The code currently plays Blackjack three times - feel free to change this number, or to run the cell multiple times.  The cell is designed for you to gain some familiarity with the output of the `generate_episode_from_limit` function.*)

In [ ]:
for i in range(3):
    print(generate_episode_from_limit(env))

Now, you are ready to write your own implementation of MC prediction.  Feel free to implement either first-visit or every-visit MC prediction; in the case of the Blackjack environment, the techniques are equivalent.

Your algorithm has three arguments:
- `env`: This is an instance of an OpenAI Gym environment.
- `num_episodes`: This is the number of episodes that are generated through agent-environment interaction.
- `generate_episode`: This is a function that returns an episode of interaction.
- `gamma`: This is the discount rate.  It must be a value between 0 and 1, inclusive (default value: `1`).

The algorithm returns as output:
- `V`: This is a dictionary where `V[s]` is the estimated value of state `s`.  For example, if your code returns the following output:
```
{(4, 7, False): -0.38775510204081631, (18, 6, False): -0.58434296365330851, (13, 2, False): -0.43409090909090908, (6, 7, False): -0.3783783783783784, ...
```
then the value of state `(4, 7, False)` was estimated to be `-0.38775510204081631`.

If you are unfamiliar with how to use `defaultdict` in Python, you are encouraged to check out [this source](https://www.accelebrate.com/blog/using-defaultdict-python/).

In [ ]:
from collections import defaultdict
from itertools import count
import numpy as np
import sys

def first_visit_mc_prediction_v(env, num_episodes, generate_episode, gamma=1.0):
    # initialize empty dictionary of lists
    returns = defaultdict(list)
    # loop over episodes
    for i_episode in range(1, num_episodes+1):
        # monitor progress
        if i_episode % 1000 == 0:
            print("\rEpisode {}/{}.".format(i_episode, num_episodes), end="")
            sys.stdout.flush()
        
        #generate an episode
        episode = generate_episode(env)
        
        visited = []
        for task_id, (state, _, _) in enumerate(episode):
            if not state in visited:
                #create a discount generator
                discounts = (gamma ** i for i in count(start=0, step=1))
                
                returns[state].append(sum([r*d for (_, _, r), d
                                           in zip(episode[task_id:], discounts)]))
                visited.append(state)        
    
    V = {k: np.mean(v) for k, v in returns.items()}
        
    return V

def every_visit_mc_prediction_v(env, num_episodes, generate_episode, gamma=1.0):
    # initialize empty dictionary of lists
    returns = defaultdict(list)
    # loop over episodes
    for i_episode in range(1, num_episodes+1):
        # monitor progress
        if i_episode % 1000 == 0:
            print("\rEpisode {}/{}.".format(i_episode, num_episodes), end="")
            sys.stdout.flush()
            
        # generate an episode
        episode = generate_episode(env)
        
        # obtain the states, actions, and rewards
        states, actions, rewards = zip(*episode)
        
        discounts = np.array([gamma ** i for i in range(len(episode) + 1)])

        for i, state in enumerate(states):
            returns[state].append(sum(rewards[i:] * discounts[:-(i+1)]))

            
    V = {k: np.mean(v) for k, v in returns.items()}
    return V

mc_prediction_v = every_visit_mc_prediction_v

Use the cell below to calculate and plot the state-value function estimate.  (_The code for plotting the value function has been borrowed from [this source](https://github.com/dennybritz/reinforcement-learning/blob/master/lib/plotting.py) and slightly adapted._)

To check the accuracy of your implementation, compare the plot below to the corresponding plot in the solutions notebook **Monte_Carlo_Solution.ipynb**.

In [ ]:
from plot_utils import plot_blackjack_values

# obtain the value function
V = mc_prediction_v(env, 500000, generate_episode_from_limit)

# plot the value function
plot_blackjack_values(V)

### Part 2: MC Prediction: Action Values

In this section, you will write your own implementation of MC prediction (for estimating the action-value function).  

We will begin by investigating a policy where the player _almost_ always sticks if the sum of her cards exceeds 18.  In particular, she selects action `STICK` with 80% probability if the sum is greater than 18; and, if the sum is 18 or below, she selects action `HIT` with 80% probability.  The function `generate_episode_from_limit_stochastic` samples an episode using this policy. 

The function accepts as **input**:
- `bj_env`: This is an instance of OpenAI Gym's Blackjack environment.

It returns as **output**:
- `episode`: This is a list of (state, action, reward) tuples (of tuples) and corresponds to $(S_0, A_0, R_1, \ldots, S_{T-1}, A_{T-1}, R_{T})$, where $T$ is the final time step.  In particular, `episode[i]` returns $(S_i, A_i, R_{i+1})$, and `episode[i][0]`, `episode[i][1]`, and `episode[i][2]` return $S_i$, $A_i$, and $R_{i+1}$, respectively.

In [ ]:
def generate_episode_from_limit_stochastic(bj_env):
    episode = []
    state = bj_env.reset()
    while True:
        probs = [0.8, 0.2] if state[0] > 18 else [0.2, 0.8]
        action = np.random.choice(np.arange(2), p=probs)
        next_state, reward, done, info = bj_env.step(action)
        episode.append((state, action, reward))
        state = next_state
        if done:
            break
    return episode

Now, you are ready to write your own implementation of MC prediction.  Feel free to implement either first-visit or every-visit MC prediction; in the case of the Blackjack environment, the techniques are equivalent.

Your algorithm has three arguments:
- `env`: This is an instance of an OpenAI Gym environment.
- `num_episodes`: This is the number of episodes that are generated through agent-environment interaction.
- `generate_episode`: This is a function that returns an episode of interaction.
- `gamma`: This is the discount rate.  It must be a value between 0 and 1, inclusive (default value: `1`).

The algorithm returns as output:
- `Q`: This is a dictionary (of one-dimensional arrays) where `Q[s][a]` is the estimated action value corresponding to state `s` and action `a`.

In [ ]:
def every_visit_mc_prediction_q(env, num_episodes, generate_episode, gamma=1.0):
    # initialize empty dictionaries of arrays
    returns = defaultdict(lambda: np.zeros(env.action_space.n))
    N = defaultdict(lambda: np.zeros(env.action_space.n))
    Q = defaultdict(lambda: np.zeros(env.action_space.n))
    # loop over episodes
    for i_episode in range(1, num_episodes+1):
        # monitor progress
        if i_episode % 1000 == 0:
            print("\rEpisode {}/{}.".format(i_episode, num_episodes), end="")
            sys.stdout.flush()
        
        #generate an episode
        episode = generate_episode(env)
        
        for task_id, (state, action, _) in enumerate(episode):
            #create a discount generator
            discounts = (gamma ** i for i in count(start=0, step=1))

            N[state][action] += 1
            returns[state][action] += sum([r*d for (_, _, r), d
                                           in zip(episode[task_id:], discounts)])
            
    
    for state in returns.keys():
        Q[state] = returns[state] / N[state]
  
    return Q

mc_prediction_q = every_visit_mc_prediction_q

Use the cell below to obtain the action-value function estimate $Q$.  We have also plotted the corresponding state-value function.

To check the accuracy of your implementation, compare the plot below to the corresponding plot in the solutions notebook **Monte_Carlo_Solution.ipynb**.

In [ ]:
# obtain the action-value function
Q = mc_prediction_q(env, 500000, generate_episode_from_limit_stochastic)

# obtain the state-value function
V_to_plot = dict((k,(k[0]>18)*(np.dot([0.8, 0.2],v)) + (k[0]<=18)*(np.dot([0.2, 0.8],v))) \
         for k, v in Q.items())

# plot the state-value function
plot_blackjack_values(V_to_plot)

### Part 3: MC Control: GLIE

In this section, you will write your own implementation of constant-$\alpha$ MC control.  

Your algorithm has three arguments:
- `env`: This is an instance of an OpenAI Gym environment.
- `num_episodes`: This is the number of episodes that are generated through agent-environment interaction.
- `gamma`: This is the discount rate.  It must be a value between 0 and 1, inclusive (default value: `1`).

The algorithm returns as output:
- `Q`: This is a dictionary (of one-dimensional arrays) where `Q[s][a]` is the estimated action value corresponding to state `s` and action `a`.
- `policy`: This is a dictionary where `policy[s]` returns the action that the agent chooses after observing state `s`.

In [ ]:
from typing import TypeVar

Policy = TypeVar('Policy')
Episode = TypeVar('Episode')
ActionValue = TypeVar('ActionValue')

In [ ]:
# NOT optimized solution
def generate_policy(Q: ActionValue, n_actions: int, e: float) -> Policy:
    policy = defaultdict(lambda: np.ones(n_actions) * 1/n_actions)
    for s in Q.keys():  
        for a in range(n_actions):
            max_action = np.argmax(Q[s]) == a
            policy[s][a] = 1 - e + e/n_actions if max_action else e/n_actions
            
    return policy

def generate_episode_from_policy(policy: Policy, n_actions: int, env: gym.core.Env) -> Episode:
    episode = []
    state = env.reset()
    while True:
        action = np.random.choice(np.arange(n_actions),
                                  p=policy[state])
        
        next_state, reward, done, info = env.step(action)
        episode.append((state, action, reward))
        state = next_state
        if done:
            break
    return episode
    

def mc_control_GLIE_unoptimized(env: gym.core.Env, num_episodes: int, gamma: float = 1.0):
    n_actions = env.action_space.n
    # initialize empty dictionaries of arrays
    Q = defaultdict(lambda: np.zeros(n_actions))
    N = defaultdict(lambda: np.zeros(n_actions))
    # loop over episodes
    for i_episode in range(1, num_episodes+1):
        # monitor progress
        if i_episode % 1000 == 0:
            print("\rEpisode {}/{}.".format(i_episode, num_episodes), end="")
            sys.stdout.flush()
            
        e = 1 / (i/10000 + 1)
        
        policy = generate_policy(Q, n_actions, e)
                
        episode = generate_episode_from_policy(policy, n_actions, env) 
        
        for task_id, (state, action, reward) in enumerate(episode):
            #create a discount generator
            discounts = (gamma ** i for i in count(start=0, step=1))
            reward = sum([r*d for (_, _, r), d in zip(episode[task_id:], discounts)])
            
            N[state][action] += 1
            Q[state][action] += (1/N[state][action] * (reward - Q[state][action]))
        
    policy = {k: np.argmax(v) for k, v in Q.items()}    
    return policy, Q

In [ ]:
# optimized solution
def generate_micro_policy(q_s: np.array, e: float, n_actions: int) -> Policy:
    if np.array_equal(q_s, np.zeros(n_actions)):
        return np.ones(n_actions) * 1/n_actions    
    
    micro_policy_s = [e/n_actions] * n_actions
    best_action = np.argmax(q_s)
    micro_policy_s[best_action] = 1 - e + e/n_actions
    
    return micro_policy_s

 
def generate_episode_from_Q(Q: np.array, e: float, n_actions: int, env: gym.core.Env) -> Episode:
    episode = []
    state = env.reset()
    
    while True:
        action = np.random.choice(np.arange(n_actions),
                                  p=generate_micro_policy(Q[state], e, n_actions))
        
        next_state, reward, done, info = env.step(action)
        episode.append((state, action, reward))
        state = next_state
        if done:
            break
    return episode


def mc_control_GLIE_optimized(env: gym.core.Env,
                              num_episodes: int,
                              gamma: float = 1.0) -> (Policy, ActionValue):
    n_actions = env.action_space.n
    # initialize empty dictionaries of arrays
    Q = defaultdict(lambda: np.zeros(n_actions))
    N = defaultdict(lambda: np.zeros(n_actions))
    # loop over episodes
    for i_episode in range(1, num_episodes+1):
        # monitor progress
        if i_episode % 1000 == 0:
            print("\rEpisode {}/{}.".format(i_episode, num_episodes), end="")
            sys.stdout.flush()
            
        e = 1.0 / (i_episode/10000 + 1)
                
        episode = generate_episode_from_Q(Q, e, n_actions, env) 
    
        for task_id, (state, action, _) in enumerate(episode):
            #create a discount generator
            discounts = (gamma ** i for i in count(start=0, step=1))
            reward = sum([r*d for (_, _, r), d in zip(episode[task_id:], discounts)])
            
            N[state][action] += 1
            Q[state][action] += (reward - Q[state][action]) / N[state][action]
            
        
    
    policy = {k: np.argmax(v) for k, v in Q.items()}
    return policy, Q

In [ ]:
mc_control_GLIE = mc_control_GLIE_optimized

Use the cell below to obtain the estimated optimal policy and action-value function.

In [ ]:
# obtain the estimated optimal policy and action-value function
policy_glie, Q_glie = mc_control_GLIE(env, 500000)

Next, we plot the corresponding state-value function.

In [ ]:
# obtain the state-value function
V_glie = dict((k,np.max(v)) for k, v in Q_glie.items())

# plot the state-value function
plot_blackjack_values(V_glie)

Finally, we visualize the policy that is estimated to be optimal.

In [ ]:
import plot_utils
from plot_utils import plot_policy

from importlib import reload
reload(plot_utils)
# plot the policy
plot_policy(policy_glie)

The **true** optimal policy $\pi_*$ can be found on page 82 of the [textbook](http://go.udacity.com/rl-textbook) (and appears below).  Compare your final estimate to the optimal policy - how close are you able to get?  If you are not happy with the performance of your algorithm, take the time to tweak the decay rate of $\epsilon$ and/or run the algorithm for more episodes to attain better results.

![True Optimal Policy](images/optimal.png)

### Part 4: MC Control: Constant-$\alpha$

In this section, you will write your own implementation of constant-$\alpha$ MC control.  

Your algorithm has four arguments:
- `env`: This is an instance of an OpenAI Gym environment.
- `num_episodes`: This is the number of episodes that are generated through agent-environment interaction.
- `alpha`: This is the step-size parameter for the update step.
- `gamma`: This is the discount rate.  It must be a value between 0 and 1, inclusive (default value: `1`).

The algorithm returns as output:
- `Q`: This is a dictionary (of one-dimensional arrays) where `Q[s][a]` is the estimated action value corresponding to state `s` and action `a`.
- `policy`: This is a dictionary where `policy[s]` returns the action that the agent chooses after observing state `s`.

(_Feel free to define additional functions to help you to organize your code._)

In [ ]:
def mc_control_alpha(env, num_episodes, alpha, gamma=1.0):
    n_actions = env.action_space.n
    # initialize empty dictionary of arrays
    Q = defaultdict(lambda: np.zeros(n_actions))
    # loop over episodes
    for i_episode in range(1, num_episodes+1):
        # monitor progress
        if i_episode % 1000 == 0:
            print("\rEpisode {}/{}.".format(i_episode, num_episodes), end="")
            sys.stdout.flush()
        
        e = 1.0 / (i_episode/12000 + 1)
                
        episode = generate_episode_from_Q(Q, e, n_actions, env) 
    
        for task_id, (state, action, _) in enumerate(episode):
            #create a discount generator
            discounts = (gamma ** i for i in count(start=0, step=1))
            reward = sum([r*d for (_, _, r), d in zip(episode[task_id:], discounts)])
            
            Q[state][action] += alpha * (reward - Q[state][action])
                   
    policy = {k: np.argmax(v) for k, v in Q.items()}
    return policy, Q

Use the cell below to obtain the estimated optimal policy and action-value function.

In [ ]:
# obtain the estimated optimal policy and action-value function
policy_alpha, Q_alpha = mc_control_alpha(env, 5000000, 0.0008, gamma=0.98)

Next, we plot the corresponding state-value function.

In [ ]:
# obtain the state-value function
V_alpha = dict((k,np.max(v)) for k, v in Q_alpha.items())

# plot the state-value function
plot_blackjack_values(V_alpha)

Finally, we visualize the policy that is estimated to be optimal.

In [ ]:
# plot the policy
plot_policy(policy_alpha)

The **true** optimal policy $\pi_*$ can be found on page 82 of the [textbook](http://go.udacity.com/rl-textbook) (and appears below).  Compare your final estimate to the optimal policy - how close are you able to get?  If you are not happy with the performance of your algorithm, take the time to tweak the decay rate of $\epsilon$, change the value of $\alpha$, and/or run the algorithm for more episodes to attain better results.

![True Optimal Policy](images/optimal.png)